## Pandas - Extracting data

In [2]:
import pandas as pd
import numpy as np
import glob

# Load data from multiple CSV files into one dataframe
df = pd.concat([pd.read_csv(f) for f in glob.glob('/dbfs/police-data/2017-*/*.csv')], ignore_index = True)

## Pandas - Encoding & Extending Data

In [4]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

# Remove junk
try:
    df = df.drop(['Context'], axis=1)
except:
    pass
df = df.dropna()

df['Last outcome category_e'] = enc.fit_transform(df['Last outcome category'])
Outcome_mapping = dict(zip(enc.classes_, range(len(enc.classes_))))

# This maps the various outcomes to 1 or 0, to a "Caught" column
def mapOutcome(val):
    if val in [0, 1, 5, 9, 19, 11, 15, 16, 17, 18, 19, 20, 21, 23]:
        return 1
    else:
        return 0
df['Caught'] = df['Last outcome category_e'].map(mapOutcome).astype(int)

df['Falls within_e'] = enc.fit_transform(df['Falls within'])
FallsWithin_mapping = dict(zip(enc.classes_, range(len(enc.classes_))))

df['Crime type_e'] = enc.fit_transform(df['Crime type'])
CrimeType_mapping = dict(zip(enc.classes_, range(len(enc.classes_))))

# Encode month by striping the string
df['Month_e'] = df['Month'].str.slice(5).astype(int)

#print(FallsWithin_mapping)
print(CrimeType_mapping)
#print(Outcome_mapping)
#df.head(20)
#print(df.shape)
#df.iloc[:5, 15]
#df.info()

# Get our training data in NumPy format
train_data = df.values

{'Drugs': 3, 'Vehicle crime': 11, 'Other theft': 5, 'Theft from the person': 10, 'Burglary': 1, 'Bicycle theft': 0, 'Possession of weapons': 6, 'Robbery': 8, 'Shoplifting': 9, 'Violence and sexual offences': 12, 'Criminal damage and arson': 2, 'Other crime': 4, 'Public order': 7}

## Scikit-learn - Training the model

In [6]:
#train_data[0:,14]

from sklearn.ensemble import RandomForestClassifier

# Use RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100)
X = train_data[0:,13:].astype(int)
y = train_data[0:,12].astype(int)
model = model.fit(X, y)

## Test

In [8]:
# Test all forces, with crime 6 = possesion of a weapon
for f, fi in FallsWithin_mapping.items():
    a = model.predict_proba([[fi, 3, 1]])[0]
    print(f, a[0])

Essex Police 0.770567276999
City of London Police 0.78841775459
Hampshire Constabulary 0.733511951824
Devon & Cornwall Police 0.798876823168
Dyfed-Powys Police 0.827176643473
Northumbria Police 0.829691861758
Greater Manchester Police 0.788936054012
North Yorkshire Police 0.63310354737
Lancashire Constabulary 0.640533821382
Dorset Police 0.62712666009
Gloucestershire Constabulary 0.879407283732
Cumbria Constabulary 0.695059523295
Derbyshire Constabulary 0.688940906327
South Yorkshire Police 0.802465627272
Thames Valley Police 0.733249719474
Hertfordshire Constabulary 0.824128478792
Suffolk Constabulary 0.775336402904
Cheshire Constabulary 0.722857198798
South Wales Police 0.571632444504
Cleveland Police 0.786105106078
Metropolitan Police Service 0.726046442664
Merseyside Police 0.672814034962
Durham Constabulary 0.699172212755
Staffordshire Police 0.490960879521
Lincolnshire Police 0.868830479899
Bedfordshire Police 0.668862099297
North Wales Police 0.778468305007
Warwickshire Police 0.789656942397
Gwent Police 0.723234241761
Humberside Police 0.602570151327
Northamptonshire Police 0.949896229764
West Yorkshire Police 0.539392501757
Surrey Police 0.689516835175
West Midlands Police 0.799335035958
Nottinghamshire Police 0.841216748202
Leicestershire Police 0.658475460555
Cambridgeshire Constabulary 0.880673588585
Kent Police 0.541934525921
West Mercia Police 0.788358862376
Avon and Somerset Constabulary 0.884250585177
Sussex Police 0.794715536914
Norfolk Constabulary 0.823859637492
Wiltshire Police 0.882728943661

## Pickle model and store in Azure storage

In [10]:
#
# Widgets are how we get values passed from a DataBricks job
#
try:
  #dbutils.widgets.text("model_version", "1.0.0")
  #dbutils.widgets.text("storage_account", "modelreg")
  #dbutils.widgets.text("model_name", "batcomputer")

  # Model version, name & storage-account is passed into job, and storage key is kept in Azure Key Vault
  STORAGE_KEY       = dbutils.secrets.get("ai-deploy-secrets", "storage-key")
  STORAGE_ACCOUNT   = dbutils.widgets.get("storage_account")
  MODEL_VERSION     = dbutils.widgets.get("model_version")
  STORAGE_CONTAINER = dbutils.widgets.get("model_name")
except:
    pass
    
#
# STORAGE_ACCOUNT value should only be set when this Notebook is invoked via a job
# So we only pickle and store in Azure blobs when running as a job
#
if 'STORAGE_ACCOUNT' in vars():
  print("Saving pickles to:", STORAGE_ACCOUNT, " / ", STORAGE_CONTAINER)
  
  # Create pickles and data lookup 
  from collections import OrderedDict
  import pickle

  lookup = OrderedDict()

  # ORDER IS IMPORTANT! This is why we use OrderedDict and create entries one by one
  lookup["Force"] = FallsWithin_mapping
  lookup["Crime"] = CrimeType_mapping

  # Create output lookup
  flags = ["getaway_proba", "busted_proba"]

  # Pickle the whole damn lot
  with open("model.pkl" , 'wb') as file:  
    pickle.dump(model, file)
    file.close()

  with open("lookup.pkl" , 'wb') as file:  
    pickle.dump(lookup, file)
    file.close()

  with open("flags.pkl" , 'wb') as file:  
    pickle.dump(flags, file)    
    file.close()

  from azure.storage.blob import BlockBlobService

  # Create the BlockBlockService that is used to call the Blob service for the storage account
  block_blob_service = BlockBlobService(account_name=STORAGE_ACCOUNT, account_key=STORAGE_KEY) 

  # Create a container
  block_blob_service.create_container(STORAGE_CONTAINER) 

  # Upload the created file, use local_file_name for the blob name
  block_blob_service.create_blob_from_path(STORAGE_CONTAINER, MODEL_VERSION + "/model.pkl", "model.pkl")
  block_blob_service.create_blob_from_path(STORAGE_CONTAINER, MODEL_VERSION + "/lookup.pkl", "lookup.pkl")
  block_blob_service.create_blob_from_path(STORAGE_CONTAINER, MODEL_VERSION + "/flags.pkl", "flags.pkl")